__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-16 15:01:46 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-16 15:01:46 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-16 15:01:48 | AdsListParser: downloader init
[INFO    ] 2022-09-16 15:01:48 | AvitoParser: init
[INFO    ] 2022-09-16 15:01:54 | AvitoParser: 77 pages for read
[INFO    ] 2022-09-16 15:01:54 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-16 15:01:58 | AdsListParser: read page 1
[INFO    ] 2022-09-16 15:02:01 | AdsListParser: read page 2
[INFO    ] 2022-09-16 15:02:05 | AdsListParser: read page 3
[INFO    ] 2022-09-16 15:02:09 | AdsListParser: read page 4
[INFO    ] 2022-09-16 15:02:14 | AdsListParser: read page 5
[INFO    ] 2022-09-16 15:02:19 | AdsListParser: read page 6
[INFO    ] 2022-09-16 15:02:23 | AdsListParser: read page 7
[INFO    ] 2022-09-16 15:02:27 | AdsListParser: read page 8
[INFO    ] 2022-09-16 15:02:34 | AdsListParser: read page 9
[INFO    ] 2022-09-16 15:02:39 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3852
3391


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
2232,2229269027,"Объявление «2-к. квартира, 74,4 м², 2/5 эт.» 6 фотографий",9300000,,"ул. Военных Строителей, 12/3","Продаётся 2 комнатная квартира общей площадью 74.4кв. М. Хороший район, в пешей доступности пляж. Развитая инфраструктура. Квартира расположена на 2 этаже 5 этажного дома. Дом Сдан. Документы Российского образца, готовы к сделке. \nПродажа возможна с исполь",52,2022-09-16 15:07:31.631801,sevastopol
864,2553443030,"Объявление «Квартира-студия, 31,5 м², 3/9 эт.» 21 фотография",4550000,,"ул. Горпищенко, 139","Дом Сдан! \n\nСемейная Ипотека, Материнский Капитал Первым Взносом! \nКвартира-студия на третьем этаже девятиэтажного дома в черновой отделке от застройщика. \nПросторная и светлая, с большой лоджией. \nВид из окна на сосновый лес! \nВ студии установлено автоном",20,2022-09-16 15:07:31.631801,sevastopol
